# Building wind suitablity layers for GRIDCERF

## 1. Downloading the data

### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


### 1.2 Download the wind potential data

Data for potential wind capacity for 80m, 110m, and 140m hub heights for current technology assumptions may be access from https://maps.nrel.gov/wind-prospector.  You can directly download the data using this link:  https://gds-files.nrelcloud.org/archive/wind-prospector.zip.  Please extract these data to the `data/gridcerf/source/wind` directory as this is where the current paths in this notebook point to.

Metadata for potential resource capacity files:  https://edacarc20.unm.edu/arcgis/rest/services/nmenergytool/EDAC_Energy/FeatureServer/0


## 2. Setup environment

### 2.1 Install GDAL

This application requires GDAL to be installed.  We will call GDAL directly from your command prompt or terminal, so please ensure that you can do so before running the following cells.  More information on how to install GDAL can be found here:  https://gdal.org/download.html


### 2.2 Install Python third-party dependencies

The following Python packages can be installed directly from the following kernel if they are not already in your kernel's environment.  If you are using conda, please use conda specific install protocol.

**NOTE**: Executing the following cell will install these Python packages over your existing versions!  


In [ ]:
!pip install -Iv numpy>=1.23.1
!pip install -Iv pandas>=1.4.3
!pip install -Iv rasterio>=1.3.4
!pip install -Iv geopandas>=0.12.2


### 2.3 Import necessary Python packages

In [7]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd


## 3. Configuration

In [27]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source wind data directory
wind_source_dir = os.path.join(gridcerf_dir, "source", "wind")

# Field in wind potential shapefiles that represent the area per polygon that is 
#  available for supporting wind turbines when considering a gross capacity factor
#  of 35%. 
target_wind_area_field = "AREA WITH "

# target fields to include when conducting rasterization
target_wind_fields = [target_wind_area_field, "geometry"]

# generate a list of all common exclusion files
common_raster_list = glob.glob(os.path.join(common_dir, "*.tif"))

# list of technology specific files to use as exclusion areas
technology_specific_list = [os.path.join(technology_specific_dir, "gridcerf_srtm_slope_20pct_or_less.tif")]

# list of suitability files to apply to the wind raster
suitability_file_list = common_raster_list + technology_specific_list


## 4. Generate wind suitability rasters

### 4.1 Functions to build suitability

In [33]:
def filter_wind_potential_shapefile(hub_height: int, 
                                    wind_source_dir: str,
                                    target_wind_area_field: str,
                                    target_wind_fields: list):
    
    # construct the target wind potential file name
    root_name = f"{hub_height}_Meter_Hub_Height_Current_Technology_"
    target_wind_file = os.path.join(wind_source_dir, root_name, f"{root_name}.shp")

    # read in wind potential shapefile as a GeoPandas data frame
    gdf = gpd.read_file(target_wind_file)[target_wind_fields]
    
    # only keep polygons having an area capable of supporting 35% GCF of >= 1km
    gdf = gdf.loc[gdf[target_wind_area_field] >= 1].copy()
    
    # set target value for rasterization
    gdf["value"] = 0
    
    # save filtered shapefile
    basename = f"gridcerf_nrel_wind_development_potential_hubheight{hub_height}m_cf35"
    output_shpfile = os.path.join(wind_source_dir, f"{basename}.shp")
    gdf[["value", "geometry"]].to_file(output_shpfile)
    
    return output_shpfile, basename


def rasterize_potential_noexclusions(input_shapefile: str, 
                                     file_basename: str,
                                     output_dir: str):

    # construct output raster name and path
    output_raster = os.path.join(output_dir, f"{file_basename}_no-exclusions.tif")

    # construct command to rasterize the shapefile using GDAL
    gdal_rasterize_cmd = f"gdal_rasterize -l {file_basename} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {input_shapefile} {output_raster}"

    # execute the GDAL command
    print(gdal_rasterize_cmd, '\n')
    os.system(gdal_rasterize_cmd)
    
    return output_raster


def construct_wind_suitability(hub_height: int,
                               input_wind_raster: str, 
                               output_suitability_dir: str,
                               suitability_file_list: list):
    
    # open input wind potential raster
    with rasterio.open(input_wind_raster) as wind_potential_raster:

        # update metadata datatype to int16
        metadata = wind_potential_raster.meta.copy()
        metadata.update({'dtype': rasterio.int16})

        # get array of potential suitability
        wind_suitability_array = wind_potential_raster.read(1)

    # apply suitability rasters
    for idx, i in enumerate(suitability_file_list):
        with rasterio.open(i) as src:
            wind_suitability_array += src.read(1)

    # reclassify to 0 (suitable), 1 (unsuitable)
    wind_suitability_array = np.where(wind_suitability_array == 0, 0, 1)

    # write aggregate tech-specific suitability layer to file
    final_raster = os.path.join(output_suitability_dir, f'gridcerf_wind_onshore_{hub_height}.tif')
    with rasterio.open(final_raster, 'w', **metadata) as dest:
        dest.write(wind_suitability_array.astype(rasterio.int16), 1)
        
    print(f"Generated wind suitability raster:  {final_raster}")
        
    return final_raster

    

### 4.2 Generate wind suitability rasters for all desired hub heights

In [34]:
# list of hub heights to process
hub_height_list = [80, 110, 140]

for hub_height in hub_height_list:
    
    print(f"Processing hub height:  {hub_height}...")

    # create a filtered shapefile containing all wind potential polygons having >= 1km of area able to support a GCF of 35%
    filtered_shapefile, basename = filter_wind_potential_shapefile(hub_height=hub_height, 
                                                                   wind_source_dir=wind_source_dir,
                                                                   target_wind_area_field=target_wind_area_field,
                                                                   target_wind_fields=target_wind_fields)

    # rasterize the shapefile into GRIDCERF's format
    raster_noexclusions = rasterize_potential_noexclusions(input_shapefile=filtered_shapefile,
                                                           file_basename=basename,
                                                           output_dir=wind_source_dir)
    
    # generate wind suitability output including suitability exclusions
    wind_suitability_raster_file = construct_wind_suitability(hub_height=hub_height,
                                                              input_wind_raster=raster_noexclusions, 
                                                              output_suitability_dir=compiled_dir,
                                                              suitability_file_list=suitability_file_list)


gdal_rasterize -l gridcerf_nrel_wind_development_potential_hubheight80m_cf35 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/wind/gridcerf_nrel_wind_development_potential_hubheight80m_cf35.shp /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/wind/gridcerf_nrel_wind_development_potential_hubheight80m_cf35_no-exclusions.tif 

0...10...20...30...40...50...60...70...80...90...100 - done.
Generated wind suitability raster:  /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/compiled/gridcerf_wind_onshore_80.tif
gdal_rasterize -l gridcerf_nrel_wind_development_potential_hubheight110m_cf35 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/source/wind/gridcerf_nrel_wind_develop

In [16]:
common_dir = 

'/Users/d3y010/repos/metarepos/vernon-etal_2022_scidata/data/gridcerf/reference'

In [23]:
for x in [i for i in os.listdir(technology_specific_dir) if ".DS_Store" not in i]:
    
    s = x.split("_")
    
    old_name = os.path.join(technology_specific_dir, x)
    new_name = os.path.join(technology_specific_dir, f"gridcerf_{'_'.join(s[1:])}")
    
    os.rename(old_name, new_name)
                            
                            

In [3]:
import os
import glob

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd


In [17]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF common data directory
common_dir = os.path.join(gridcerf_dir, "common")

# GRIDCERF technology-specific directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# source wind data directory
wind_source_dir = os.path.join(gridcerf_dir, "source", "wind")

# exclusion data for wind dir
exclusion_dir = os.path.join(data_dir, 'wind', 'exclusions')

# directory where downloaded data was decompressed
download_dir = os.path.join(data_dir, 'wind', 'nrel')

# processed shapefile directory
wind_shp_dir = os.path.join(data_dir, 'wind', 'shp')

# directory to write potential capacity rasters to
potential_dir = os.path.join(data_dir, 'wind', 'potential_suitability')

# where to write the final suitability outputs to
output_dir = os.path.join(data_dir, 'wind')


In [3]:
target = os.path.join(wind_source_dir, "80_Meter_Hub_Height_Current_Technology_", "80_Meter_Hub_Height_Current_Technology_.shp")

area_field = "AREA WITH "

fields = [area_field, "AREA (KM^2", "geometry"]

gdf = gpd.read_file(target)#[fields]


In [4]:
gdf.head()

,LATITUDE,LONGITUDE,STATE,REGION,AREA WITH,AREA WI_01,AREA WI_02,AREA (KM^2,geometry
0,48.3631,-119.3485,Washington,West,10.60,3.52,2.80,420.252008,"POLYGON ((-119.25220 48.47656, -119.17886 48.3..."
1,47.9232,-121.4517,Washington,West,0.03,0.00,0.00,733.111571,"POLYGON ((-121.40695 47.83655, -121.53621 47.8..."
2,47.4343,-123.5047,Washington,West,2.95,2.49,1.43,632.391386,"POLYGON ((-123.33263 47.38107, -123.58215 47.3..."
3,48.8966,-123.1409,Washington,West,9.16,0.00,0.00,9.936960,"POLYGON ((-123.04761 49.00212, -123.03339 48.9..."
4,47.6232,-122.7409,Washington,West,5.57,1.93,1.26,326.908299,"MULTIPOLYGON (((-122.65686 47.73984, -122.6564..."


In [5]:
gdf = gdf.loc[gdf[area_field] > 1].copy()

In [6]:
gdf['value'] = 0

In [7]:
gdf.shape

(16901, 10)

In [8]:
gdf.to_file("/Users/d3y010/Desktop/file0.shp")

In [39]:
gdf.head()

,AREA WITH,AREA (KM^2,geometry,value
0,10.60,420.252008,"POLYGON ((-119.25220 48.47656, -119.17886 48.3...",0
2,2.95,632.391386,"POLYGON ((-123.33263 47.38107, -123.58215 47.3...",0
3,9.16,9.936960,"POLYGON ((-123.04761 49.00212, -123.03339 48.9...",0
4,5.57,326.908299,"MULTIPOLYGON (((-122.65686 47.73984, -122.6564...",0
6,374.54,414.323646,"POLYGON ((-118.79886 47.41812, -118.72818 47.2...",0


In [34]:
file_name = f'cerf_nrel_wind_development_potential_hubheight80m_cf30.shp'
output_file = os.path.join("/Users/d3y010/Desktop", file_name)

# write output
gdf[['value', 'geometry']].to_file(output_file)


In [36]:
basename = os.path.basename(output_file)
base_noext = os.path.splitext(basename)[0]

output_raster = os.path.join("/Users/d3y010/Desktop", f"{base_noext}.tif")

gdal_rasterize_cmd = f"gdal_rasterize -l {base_noext} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {output_file} {output_raster}"

print(gdal_rasterize_cmd, '\n')

os.system(gdal_rasterize_cmd)
    

gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight80m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/Desktop/cerf_nrel_wind_development_potential_hubheight80m_cf30.shp /Users/d3y010/Desktop/cerf_nrel_wind_development_potential_hubheight80m_cf30.tif 

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [7]:
# get cerf CRS from template raster
with rasterio.open(template_raster) as src:
    target_crs = src.crs


### Extract desired data from downloaded shapefiles and save to new files

In [8]:
%%time

# list of hub heights to process
hub_height_list = ['080', '110', '140']

for hub_height in hub_height_list:

    # input shapefile
    f = os.path.join(download_dir, f'pot_wind_cap_{hub_height}_current', f'pot_wind_cap_{hub_height}_current.shp')

    # only keep necessary fields
    gdf = gpd.read_file(f)[['area_km2', 'a30', 'geometry']].to_crs(target_crs)

    # get polygons with at least 1 km2 of available development area at 30% capacity
    gdx = gdf.loc[gdf['a30'] > 1].copy()

    # assign suitability value
    gdx['value'] = 0
    
    file_name = f'cerf_nrel_wind_development_potential_hubheight{hub_height}m_cf30.shp'
    output_file = os.path.join(wind_shp_dir, file_name)
    
    # write output
    gdx[['value', 'geometry']].to_file(output_file)


CPU times: user 27.8 s, sys: 615 ms, total: 28.4 s
Wall time: 28.7 s


### Rasterize shapefiles for potential

In [10]:
%%time 

shp_list = glob.glob(os.path.join(wind_shp_dir, 'cerf_nrel_wind_development_potential_hubheight*m_cf30.shp'))

for i in shp_list:
        
    basename = os.path.basename(i)
    base_noext = os.path.splitext(basename)[0]
    
    output_raster = os.path.join(potential_dir, f"{base_noext}.tif")

    gdal_rasterize_cmd = f"gdal_rasterize -l {base_noext} -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff {i} {output_raster}"

    print(gdal_rasterize_cmd, '\n')
    
    os.system(gdal_rasterize_cmd)


gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight140m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/suitability/data/wind/shp/cerf_nrel_wind_development_potential_hubheight140m_cf30.shp /Users/d3y010/projects/cerf/suitability/data/wind/potential_suitability/cerf_nrel_wind_development_potential_hubheight140m_cf30.tif 

gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight080m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.8355 -1389065.2005 2287447.1645 1609934.7995 -ot Int16 -of GTiff /Users/d3y010/projects/cerf/suitability/data/wind/shp/cerf_nrel_wind_development_potential_hubheight080m_cf30.shp /Users/d3y010/projects/cerf/suitability/data/wind/potential_suitability/cerf_nrel_wind_development_potential_hubheight080m_cf30.tif 

gdal_rasterize -l cerf_nrel_wind_development_potential_hubheight110m_cf30 -a value -tr 1000.0 1000.0 -init 1.0 -te -2405552.83

### Apply exclusions

In [13]:
%%time 

# non-potential suitability data that applies to wind
exclusion_files = [os.path.join(exclusion_dir, i) for i in os.listdir(exclusion_dir) if os.path.splitext(i)[-1] in ('.tif', '.img')]

# combine exclusion layers into a single array
for idx, i in enumerate(exclusion_files):
    
    with rasterio.open(i) as src:
        
        if idx == 0:
            arr = src.read(1)
            
        else:
            arr += src.read(1)


CPU times: user 347 ms, sys: 49.9 ms, total: 397 ms
Wall time: 766 ms


In [14]:
# apply to each potential layer
potential_rasters = glob.glob(os.path.join(potential_dir, 'cerf_nrel_wind_development_potential_hubheight*m_cf30.tif'))

for i in potential_rasters:

    with rasterio.open(i) as src:

        # update metadata datatype to int16
        metadata = src.meta.copy()
        metadata.update({'dtype': rasterio.int16})
        
        # get array of potential suitability
        arx = src.read(1)
        
        # apply exclusions
        arx += arr
        
        # reclassify to 0 (suitable), 1 (unsuitable)
        arx = np.where(arx == 0, 0, 1)
        
        # write aggregate tech-specific suitability layer to file
        hub_height_str = i.split('_')[-2]
        final_raster = os.path.join(output_dir, f'cerf_wind_onshore_{hub_height_str}.tif')
        
        with rasterio.open(final_raster, 'w', **metadata) as dest:
            dest.write(arx.astype(rasterio.int16), 1)
